In [1]:
import numpy as np
import copy
from colorama import Fore, Back, Style

In [2]:
class colors:
    reset='\033[0m'
    bold='\033[01m'
    disable='\033[02m'
    underline='\033[04m'
    reverse='\033[07m'
    strikethrough='\033[09m'
    invisible='\033[08m'
    class fg:
        black='\033[30m'
        red='\033[31m'
        green='\033[32m'
        orange='\033[33m'
        blue='\033[34m'
        purple='\033[35m'
        cyan='\033[36m'
        lightgrey='\033[37m'
        darkgrey='\033[90m'
        lightred='\033[91m'
        lightgreen='\033[92m'
        yellow='\033[93m'
        lightblue='\033[94m'
        pink='\033[95m'
        lightcyan='\033[96m'
    class bg:
        black='\033[40m'
        red='\033[41m'
        green='\033[42m'
        orange='\033[43m'
        blue='\033[44m'
        purple='\033[45m'
        cyan='\033[46m'
        lightgrey='\033[47m'

In [3]:
def print_board(board):
    print(colors.bg.black + colors.fg.lightblue+ "   A   B   C   D   E   F   G   H   ")
    count = 1
    for row in board:
        print(colors.fg.lightblue + str(count), end="  ")
        count+=1
        for cell in row:
            if cell == 0:
                print(colors.fg.green + "█", end="   ")
            if cell == 1:
                print(colors.fg.lightgrey + "█", end="   ")
            if cell == -1:
                print(colors.fg.black + "█", end="   ")
            if cell == 3:
                print(colors.fg.green + "░", end="   ")
        print("")

In [4]:
class State:
    def __init__(self, *input):
        self.origin = []
        self.destin = None
        self.heuristic = 0
        self.pieces_taken = []
        self.board = input[0]
        self.utility = 0
        if len(input)>1:
            self.origin = input[0]
            self.destin = input[1]
            self.heuristic = input[2]
            self.pieces_taken = input[3]
            self.board = input[4]
        
    def list_actions(self,player):
        token = 1
        if player == "min" : token = -1
        enemy_indexes = np.where(self.board == token)
        enemy_coordinates= list(zip(enemy_indexes[0], enemy_indexes[1]))
        return enemy_coordinates

    def print_board_changes(self):
        if type(self.origin) is list:
            player_piece = self.board[self.origin[0][0]][self.origin[0][1]]
        else:
            player_piece = self.board[self.origin[0]][self.origin[1]]
        for piece in self.pieces_taken:
            self.board[piece[0]][piece[1]] = player_piece
        self.board[self.destin[0]][self.destin[1]] = player_piece

    def can_move_up(self, piece, board, player_piece):# 1 -> y  0 -> x
        if piece[1] == 0: return False
        if board[piece[0]][piece[1]-1] == player_piece: return False
        if board[piece[0]][piece[1]-1] == player_piece*-1: return "Enemy"
        return "Empty" #enemy found?

    def can_move_down(self, piece, board, player_piece):
        if piece[1] == 7: return False
        if board[piece[0]][piece[1]+1] == player_piece: return False
        if board[piece[0]][piece[1]+1] == player_piece*-1: return "Enemy"
        return "Empty"
    
    def can_move_left(self, piece, board, player_piece):
        if piece[0] == 0: return False
        if board[piece[0]-1][piece[1]] == player_piece: return False
        if board[piece[0]-1][piece[1]] == player_piece*-1: return "Enemy"
        return "Empty"

    def can_move_right(self, piece, board, player_piece):
        if piece[0] ==7 : return False
        if board[piece[0]+1][piece[1]] == player_piece: return False
        if board[piece[0]+1][piece[1]] == player_piece*-1: 
            return "Enemy"
        return "Empty"

hola
